## 内容
```meting_0107```の改定前バージョン

### 参考文献
http://tombo.sub.jp/doc/esj55/MCMC.pdf

In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
#valiable
warnings.resetwarnings()

warnings.simplefilter('error')
num_select = 1

MCMC_step = 10000
MCMC_step_each = 5

Max_temper = 10000
T = [j for j in range(1, Max_temper+1)]
len_T = len(T)

b = [1/ l for l in T]

In [3]:
X = pd.read_table('training_markers.txt', header=None,  sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）


data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
y= data_yields.T.iloc[0]

num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

In [4]:
#type(data_yields) #pandas.core.frame.DataFrame

type(X) #pandas.core.frame.DataFrame
X.shape #(20, 20)

type(y) #pandas.core.series.Series
y.shape #(20,)

#X

(20,)

In [5]:
#特徴量の分散や同じ特徴量が存在するのかを確認

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0)
sel.fit(X)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
print(sum(sel.get_support())) #20、すなわち同じ特徴量は存在していないことがわかった

20


In [6]:
def selection(num_select, X, y):
    # num_select個の特徴量を選択
    #print(f_regression(X,y))
    #print(np.linalg.norm(y))
    selector = SelectKBest(score_func=f_regression, k=num_select) #SelectKBest
    selector.fit(X, y)
    mask = selector.get_support() #各特徴量を選択したか否かのmaskを取得
    return mask

In [7]:
"""
0-division error occurs when all values in y are 0 or 1
reason: y = y - np.mean(y), corr /= np.linalg.norm(y)
"""
mask = selection(num_select, X, y)
print(mask)

[False False False False False False False False False False False False
 False False False False False False False  True]


In [8]:
"""
def each_energy(mask, y):
    y_list = y.values.tolist()
    energy_find_dic = {}#bestな特徴量:それを選んだときのエネルギー基準値（０となる）
    for i in range(len(mask)):
        if mask[i]==True:
            energy_find_trait = np.dot(np.array(X[:][i]).T, np.array(y_list))
            energy_find_dic[i]=energy_find_trait
    return sum(energy_find_dic.values())
    

def each_energy(mask, y):
    energy_trait_sum = 0
    for i in range(len(mask)):
        if mask[i]==True:
            energy_trait_sum += np.dot(np.array(X[:][i]).T, np.array(y_list))
    return energy_trait_sum
"""

def each_energy(every_mask, every_y): #毎回のmaskとyが引数、合致度合いの計算を行う
    energy_trait_sum = 0
    for i in range(len(every_mask)):
        if every_mask[i]==mask[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [9]:
"""
argument: s is integer
procedure: convert s from 0 to 1 or 1 to 0
return: changed value
"""
def y_renewal(s: int):
    return 1-s

In [10]:
def Montecarlo(T, num_select, MCMC_step_each, E_first, y):    
    E = []
    E.append(E_first)
    
    for j in range(MCMC_step_each):    
        i = random.randrange(num_sample)
        ori_yi = y[i]
        y[i] = y_renewal(y[i])
        if np.all(y == 1) or np.all(y == 0):
            continue
        mask_new = selection(num_select, X, y)
        E_now = selected_X_energy - each_energy(mask_new, y) #必ず0以上
        E_last = E[-1]
        r = math.exp(-(E_now-E_last)/T) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R > r :
            y[i] = ori_yi #元に戻す       
        else:
            E.append(E_now)
            y_list = y.values.tolist()
    return E, y_list #エネルギーのリスト、最後のyだけ

In [17]:
E_lists = []
Y_lists = []

mask_ori = selection(num_select, X, y)
selected_X_energy = 20 #each_energy(mask_ori, y)なので同一⇒特徴量だけポイントが入る

for k in range(1, len_T):
    E, y_list = Montecarlo(T[k], num_select, MCMC_step_each, 0, y)
    E_lists.append(E)
    Y_lists.append(y_list)

del E_lists[0]#元の条件削除
del Y_lists[0]

i = random.randrange(1, len_T)
r = exp((b[i]-b[i-1])* (E_lists[i][-1]-E_lists[i-1][-1]))
if 1 <= r:
    E_lists[i-1][-1], E_lists[i][-1] = E_lists[i][-1], E_lists[i-1][-1]


for _ in range(MCMC_step/MCMC_step_each - 1):

    for k in range(1, len_T):
        E, y_list = Montecarlo(T[k], num_select, MCMC_step_each, E_lists[k][-1], Y_lists[k])
        E_lists.append(E)
        Y_lists.append(y_list)

    del E_lists[0, len_T]#1つ前のループで作られたリストの削除
    del Y_lists[0, len_T]

    i = random.randrange(1, len_T)
    r = exp((b[i]-b[i-1])* (E_lists[i][-1]-E_lists[i-1][-1]))
    if 1 <= r:
        E_lists[i-1][-1], E_lists[i][-1] = E_lists[i][-1], E_lists[i-1][-1]

for i in range(len(E_lists)):
    print(E_lists[i][-1])




"""
E, y_go = Montecarlo(T,num_select,MCMC_step)
#print(E)
#print(len(E)==len(y_go))
#print(y_go)
result_df = pd.DataFrame(y_go)
result_df['Energy'] = E
result_df.to_csv("simulated_annealing_trial1219.csv", sep=",") #yが一要素も変わっていない
print(result_df)

E_Xaxis = [i for i in range(len(E))]
plt.plot(E_Xaxis, E)
plt.show()
"""

RuntimeWarning: divide by zero encountered in true_divide